# Transform survey SPSS file into fact and dimension tables for the data warehousing

### Brief information about the structure of SPSS data:

IBM® SPSS® Statistics data files are files specifically formatted for use by IBM SPSS Statistics, containing both data and the metadata (dictionary) that define the data.

The basic structure of IBM SPSS Statistics data files is similar to a database table:

Rows (records) are cases. Each row represents a case or an observation. For example, each individual respondent to a questionnaire is a case.
Columns (fields) are variables. Each column represents a variable or characteristic that is being measured. For example, each item on a questionnaire is a variable.

IBM SPSS Statistics data files also contain metadata that describes and defines the data contained in the file. This descriptive information is called the dictionary. The information contained in the dictionary includes:

Variable names and descriptive variable labels (VARIABLE LABELS command).
Descriptive values labels (VALUE LABELS command).
Missing values definitions (MISSING VALUES command).
Print and write formats (FORMATS command).

### About the survey:

The survey data can be categorized into three dimension tables: response, questions, answers, and users tables. 

The response dimension table contains all the details for each response, while the questions and answers tables provide information about the specific questions asked in the survey and the possible answers. In this survey, there are question groups that differ in topics but share the same answer options. These groups are organized using group IDs and unique IDs for each question and answer. To link the questions and answers in the data warehouse, compound keys are used. 

It's important to note that the user table is not generated in this work, and any user-related questions are not processed.

Some imports:

In [21]:
import pandas as pd
import pyreadstat
import datetime
import re
import os

Read the `sample_data.sav` file.

In [22]:
spss_data, meta = pyreadstat.read_sav(os.path.join(os.getcwd(), "sample_data.sav"))

Extract meta data for questions dimension table

In [23]:
questions = pd.DataFrame({'question_column': meta.column_names, 'question_label': meta.column_labels})
questions

,question_column,question_label
0,responseid,responseid
1,respid,respid
2,status,status
3,interview_start,interview_start
4,interview_end,interview_end
...,...,...
260,Q11_Coded_8,Q11_Coded: Which countries or international or...
261,Q12_Coded_1,Q12_Coded: Which country or international orga...
262,Q12_Coded_2,Q12_Coded: Which country or international orga...
263,Q12_Coded_3,Q12_Coded: Which country or international orga...


Extract meta data for answers dimension table

In [24]:
answer_list = []
for k, v in meta.variable_value_labels.items():
    for kk, vv in v.items():
        answer_list.append([k, kk, vv])
answer = pd.DataFrame(answer_list, columns=["question_column", "id", "description"])
answer

,question_column,id,description
0,l,9.0,English
1,l,1046.0,Portuguese (Brazil)
2,l,4106.0,Spanish (Guatemala)
3,l,5130.0,Spanish (Costa Rica)
4,l,6154.0,Spanish (Panama)
...,...,...,...
1090,Q12_Coded_4,17.0,Turkey
1091,Q12_Coded_4,18.0,South Africa
1092,Q12_Coded_4,19.0,Other neighbouring country
1093,Q12_Coded_4,20.0,Other country or organisation


The dimension table for responses can be enhanced by adding additional columns related to time such as day, year, and so on.

In [25]:
spss_data.columns.tolist()
dim_response = spss_data[['interview_start', 'interview_end', 'last_touched', 'lastcomplete']].applymap(lambda x: datetime.datetime.fromtimestamp(x-12219379200))
dim_response[['responseid', 'surveylengthinminutes', 'status']] = spss_data[['responseid', 'surveylengthinminutes', 'status']]
dim_response = dim_response[['status'] + [col for col in dim_response.columns if col != 'status']]
dim_response = dim_response[['responseid'] + [col for col in dim_response.columns if col != 'responseid']]
dim_response["responseid"] = dim_response["responseid"].astype("int")
dim_response['year'] = dim_response['interview_start'].dt.year
dim_response['month'] = dim_response['interview_start'].dt.month
dim_response['day'] = dim_response['interview_start'].dt.day
dim_response['hour'] = dim_response['interview_start'].dt.hour
dim_response

,responseid,status,interview_start,interview_end,last_touched,lastcomplete,surveylengthinminutes,year,month,day,hour
0,5773,complete,2023-05-26 20:18:01,2023-05-26 20:26:07,2023-05-26 20:26:08,2023-05-26 20:26:07,8.10,2023,5,26,20
1,5774,complete,2023-05-26 20:18:25,2023-05-26 20:26:32,2023-05-26 20:26:32,2023-05-26 20:26:32,8.12,2023,5,26,20
2,5775,complete,2023-05-26 20:18:25,2023-05-26 20:43:51,2023-05-26 20:43:51,2023-05-26 20:43:51,25.43,2023,5,26,20
3,5777,complete,2023-05-26 20:18:34,2023-05-26 20:27:15,2023-05-26 20:27:15,2023-05-26 20:27:15,8.68,2023,5,26,20
4,5778,complete,2023-05-26 20:19:46,2023-05-26 20:31:54,2023-05-26 20:31:54,2023-05-26 20:31:54,12.13,2023,5,26,20
...,...,...,...,...,...,...,...,...,...,...,...
8077,4482,complete,2023-05-16 01:46:11,2023-05-16 02:15:46,2023-05-16 02:15:46,2023-05-16 02:15:46,29.58,2023,5,16,1
8078,4583,complete,2023-05-16 10:20:44,2023-05-16 10:27:52,2023-05-16 10:27:52,2023-05-16 10:27:52,7.13,2023,5,16,10
8079,4919,complete,2023-05-16 20:12:51,2023-05-16 20:34:31,2023-05-16 20:34:31,2023-05-16 20:34:31,21.67,2023,5,16,20
8080,5233,complete,2023-05-17 01:25:46,2023-05-17 01:36:12,2023-05-17 01:36:12,2023-05-17 01:36:12,10.43,2023,5,17,1


Store the dimension table for responses:

In [26]:
dim_response.to_excel(os.path.join(os.getcwd(), "DimResponse.xlsx"), engine='xlsxwriter', index=False)

The columns that have been used need to be removed from the data table and this table will be transformed into a fact table.

In [27]:
spss_data = spss_data.drop(columns=['status', 'interview_start', 'interview_end', 'last_touched', 'lastcomplete', 'surveylengthinminutes'])
spss_data

,responseid,respid,l,userid,language,country,participate,q0,q1,q1na_98,...,Q11_Coded_3,Q11_Coded_4,Q11_Coded_5,Q11_Coded_6,Q11_Coded_7,Q11_Coded_8,Q12_Coded_1,Q12_Coded_2,Q12_Coded_3,Q12_Coded_4
0,5773.0,120002.0,12298.0,EC00002,12298.0,7.0,1.0,NaN,32.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN
1,5774.0,120005.0,12298.0,EC00005,12298.0,7.0,1.0,NaN,33.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
2,5775.0,120004.0,12298.0,EC00004,12298.0,7.0,1.0,NaN,41.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,5777.0,120006.0,12298.0,EC00006,12298.0,7.0,1.0,NaN,24.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
4,5778.0,120007.0,12298.0,EC00007,12298.0,7.0,1.0,NaN,49.0,0.0,...,3.0,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,4482.0,50821.0,15370.0,PA10821,15370.0,3.0,1.0,NaN,65.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
8078,4583.0,1037.0,1046.0,BR01037,1046.0,2.0,1.0,NaN,65.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
8079,4919.0,70843.0,14346.0,UR10843,14346.0,4.0,1.0,NaN,65.0,0.0,...,19.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
8080,5233.0,70883.0,14346.0,UR10883,14346.0,4.0,1.0,NaN,65.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


DimAnswers table:

Select the questions without open answers and without country-specificity.

In [28]:
answer["qu"] = answer["question_column"].str.extract(r'(^q\d{1,}_\d{1,}$|^Q\d{1,}_Coded_\d{1,}$)')
answer_part = answer[answer["qu"].isna() == False].drop(columns="qu")
answer = answer[answer["qu"].isna()].drop(columns="qu")
answer_part

,question_column,id,description
256,q7_1,1.0,Totally agree
257,q7_1,2.0,Tend to agree
258,q7_1,3.0,Tend to disagree
259,q7_1,4.0,Totally disagree
260,q7_1,99.0,Don’t know
...,...,...,...
1090,Q12_Coded_4,17.0,Turkey
1091,Q12_Coded_4,18.0,South Africa
1092,Q12_Coded_4,19.0,Other neighbouring country
1093,Q12_Coded_4,20.0,Other country or organisation


Generate QuestionGroupID from question_column.

In [29]:
answer_part["qu"] = answer_part["question_column"].str.extract(r'(\d{1,}_)')
answer_part["QuestionGroupID"] = answer_part["qu"].str.extract(r'(\d{1,})').astype('int')
answer_part = answer_part.drop(columns="qu")
answer_part.dtypes
answer_part

,question_column,id,description,QuestionGroupID
256,q7_1,1.0,Totally agree,7
257,q7_1,2.0,Tend to agree,7
258,q7_1,3.0,Tend to disagree,7
259,q7_1,4.0,Totally disagree,7
260,q7_1,99.0,Don’t know,7
...,...,...,...,...
1090,Q12_Coded_4,17.0,Turkey,12
1091,Q12_Coded_4,18.0,South Africa,12
1092,Q12_Coded_4,19.0,Other neighbouring country,12
1093,Q12_Coded_4,20.0,Other country or organisation,12


Create DimAnswers table.

In [30]:
answer_part_aggregate = answer_part.groupby(["QuestionGroupID", "description", "id"]).count()
answer_part_aggregate = answer_part_aggregate.reset_index().drop(columns="question_column")
answer_part_aggregate = answer_part_aggregate.rename(columns={"id": "AnswerID", "description": "AnswerName"})
answer_part_aggregate = answer_part_aggregate[["QuestionGroupID", "AnswerID", "AnswerName"]]
answer_part_aggregate["AnswerID"] = answer_part_aggregate["AnswerID"].astype('int')
answer_part_aggregate

,QuestionGroupID,AnswerID,AnswerName
0,7,99,Don’t know
1,7,2,Tend to agree
2,7,3,Tend to disagree
3,7,1,Totally agree
4,7,4,Totally disagree
...,...,...,...
87,30,2,No
88,30,99,Rather not say
89,30,1,Yes
90,31,0,Not Selected


Store the dimension table for answers:

In [31]:
answer_part_aggregate.to_excel(os.path.join(os.getcwd(), "DimAnswers.xlsx"), engine='xlsxwriter', index=False)

Select the questions that are not related to the users

In [32]:
questions["qu"] = questions["question_column"].str.extract(r'(^q\d{1,}_\d{1,}$|^Q\d{1,}_Coded_\d{1,}$)')
questions[questions["qu"].isna() == False]
questions_part = questions[questions["qu"].isna() == False].drop(columns="qu")
questions = questions[questions["qu"].isna()].drop(columns="qu")
questions_part

,question_column,question_label
52,q7_1,How strongly do you agree or disagree with eac...
53,q7_2,How strongly do you agree or disagree with eac...
54,q7_3,How strongly do you agree or disagree with eac...
55,q7_4,How strongly do you agree or disagree with eac...
56,q7_5,How strongly do you agree or disagree with eac...
...,...,...
260,Q11_Coded_8,Q11_Coded: Which countries or international or...
261,Q12_Coded_1,Q12_Coded: Which country or international orga...
262,Q12_Coded_2,Q12_Coded: Which country or international orga...
263,Q12_Coded_3,Q12_Coded: Which country or international orga...


Generate the IDs and DimQuestions table.

In [33]:
questions_part["QuestionID"] = questions_part["question_column"].str.extract(r'(\d{1,}$)').astype('int')
questions_part["qu"] = questions_part["question_column"].str.extract(r'(\d{1,}_)')
questions_part["QuestionGroupID"] = questions_part["qu"].str.extract(r'(\d{1,})').astype('int')
questions_part = questions_part.drop(columns="qu")
questions_part = questions_part[["QuestionGroupID", "QuestionID", "question_label"]]
questions_part

,QuestionGroupID,QuestionID,question_label
52,7,1,How strongly do you agree or disagree with eac...
53,7,2,How strongly do you agree or disagree with eac...
54,7,3,How strongly do you agree or disagree with eac...
55,7,4,How strongly do you agree or disagree with eac...
56,7,5,How strongly do you agree or disagree with eac...
...,...,...,...
260,11,8,Q11_Coded: Which countries or international or...
261,12,1,Q12_Coded: Which country or international orga...
262,12,2,Q12_Coded: Which country or international orga...
263,12,3,Q12_Coded: Which country or international orga...


Store the dimension table for questions:

In [34]:
questions_part.to_excel(os.path.join(os.getcwd(), "DimQuestions.xlsx"), engine='xlsxwriter', index=False)

There are weights in the data table that need to be kept in the fact table. For this purpose, we can save weights in a separate table and add them to the fact table later.

In [35]:
spss_data_weight = spss_data[["responseid", "WEIGHT"]]
spss_data_weight

,responseid,WEIGHT
0,5773.0,0.992684
1,5774.0,0.984114
2,5775.0,1.003862
3,5777.0,1.001350
4,5778.0,1.003862
...,...,...
8077,4482.0,1.000593
8078,4583.0,1.006877
8079,4919.0,1.106502
8080,5233.0,1.106502


Select the necessary columns and create the fact table.

In [36]:
spss_data_t = spss_data.T
spss_data_t = spss_data_t.reset_index()
spss_data_t["new_index"] = spss_data_t["index"].str.extract(r'(^q\d{1,}_\d{1,}$|^Q\d{1,}_Coded_\d{1,}$|responseid)')
spss_data_t = spss_data_t[spss_data_t["new_index"].isna() == False]
spss_data_t = spss_data_t.set_index("new_index")
spss_data_t = spss_data_t.drop(columns="index")
spss_data_part = spss_data_t.T
spss_data_part

new_index,responseid,q7_1,q7_2,q7_3,q7_4,q7_5,q7_6,q7_7,q7_8,q8_1,...,Q11_Coded_3,Q11_Coded_4,Q11_Coded_5,Q11_Coded_6,Q11_Coded_7,Q11_Coded_8,Q12_Coded_1,Q12_Coded_2,Q12_Coded_3,Q12_Coded_4
0,5773.0,4.0,1.0,4.0,4.0,1.0,4.0,4.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,NaN
1,5774.0,4.0,2.0,4.0,4.0,2.0,4.0,4.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
2,5775.0,3.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
3,5777.0,4.0,2.0,4.0,4.0,1.0,3.0,3.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
4,5778.0,2.0,2.0,4.0,4.0,3.0,4.0,1.0,3.0,0.0,...,3.0,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,4482.0,4.0,1.0,4.0,3.0,1.0,4.0,3.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
8078,4583.0,1.0,1.0,4.0,4.0,1.0,4.0,4.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN
8079,4919.0,2.0,1.0,3.0,2.0,1.0,3.0,3.0,1.0,1.0,...,19.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
8080,5233.0,4.0,2.0,1.0,1.0,1.0,2.0,4.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


Construct the fact table

In [37]:
spss_data_part = spss_data_part.set_index("responseid")
fact = pd.DataFrame(columns=["QuestionGroupID", "QuestionID", "AnswerValue", "Weight"])
for col in spss_data_part:
    a = pd.DataFrame(spss_data_part, columns=[col])
    a = a.rename(columns={col: "AnswerValue"})
    a["QuestionGroupID"] = re.search(r'(^\d{1,})', col[1:]).group(0)
    a["QuestionID"] = re.search(r'(\d{1,}$)', col[1:]).group(0)
    a = a[["QuestionGroupID", "QuestionID", "AnswerValue"]]
    a["Weight"] = spss_data_weight["WEIGHT"]
    fact = pd.concat([fact, a], axis=0)
fact["QuestionID"] = fact["QuestionID"].astype('int')
fact["QuestionGroupID"] = fact["QuestionID"].astype('int')
fact = fact.reset_index().rename(columns={"index": "responseid"})
fact

,responseid,QuestionGroupID,QuestionID,AnswerValue,Weight
0,5773.0,1,1,4.0,1.015773
1,5774.0,1,1,4.0,0.868821
2,5775.0,1,1,3.0,1.080486
3,5777.0,1,1,4.0,0.808805
4,5778.0,1,1,2.0,0.997466
...,...,...,...,...,...
1236541,4482.0,4,4,NaN,1.035829
1236542,4583.0,4,4,NaN,1.038511
1236543,4919.0,4,4,NaN,0.988283
1236544,5233.0,4,4,NaN,1.012359


Store the fact table as parquet file:

In [101]:
fact.to_parquet(os.path.join(os.getcwd(), "FactSurvey.parquet"), engine='fastparquet')